In [33]:
from pathlib import Path
import numpy as np
from pprint import pprint

from skellymodels.experimental.model_redo.managers.human import Human
from skellymodels.experimental.model_redo.fmc_anatomical_pipeline.calculate_center_of_mass import calculate_center_of_mass_from_trajectory
from skellymodels.experimental.model_redo.fmc_anatomical_pipeline.enforce_rigid_bones import enforce_rigid_bones_from_trajectory

Input the path to raw 3D data file (as would come out of SkellyForge) 

In [34]:
path_to_data = Path(r"C:\Users\aaron\FreeMocap_Data\recording_sessions\freemocap_sample_data\output_data\raw_data\mediapipe_3dData_numFrames_numTrackedPoints_spatialXYZ.npy")
data = np.load(path_to_data)


This is the new model info that's loaded from the YAML. Take a look  the model info at some point. It's very much in prototype.

In [35]:
from skellymodels.experimental.model_redo.tracker_info.model_info import MediapipeModelInfo
model_info = MediapipeModelInfo()

This is all you need to create a Human model. There's no data loaded in the model itself yet, but it has all the proper structures (tracked points/virtual points/all the anatomical definitions) that we currently have in the model data loaded for each aspect. 

In [36]:
human = Human(
            name="human_one", 
            model_info=model_info
            )
pprint([human.aspects])


[{'body': Aspect: body
  Anatomical Structure:
  33 tracked points
  4 virtual markers
  22 segments
  14 center of mass definitions
  16 joint hierarchies
  Trajectories: No trajectories
  Metadata: : {'tracker_type': 'mediapipe'}

,
  'face': Aspect: face
  Anatomical Structure:
  478 tracked points
  No virtual markers
  No segment connections
  No center of mass definitions
  No joint hierarchy
  Trajectories: No trajectories
  Metadata: : {'tracker_type': 'mediapipe'}

,
  'left_hand': Aspect: left_hand
  Anatomical Structure:
  21 tracked points
  No virtual markers
  No segment connections
  No center of mass definitions
  No joint hierarchy
  Trajectories: No trajectories
  Metadata: : {'tracker_type': 'mediapipe'}

,
  'right_hand': Aspect: right_hand
  Anatomical Structure:
  21 tracked points
  No virtual markers
  No segment connections
  No center of mass definitions
  No joint hierarchy
  Trajectories: No trajectories
  Metadata: : {'tracker_type': 'mediapipe'}

}]


For the Human class, the aspects are already set (body/left hand/right hand/face) and you can access them as properties of the class

In [37]:
#For example if you want to access the marker names for an aspect, you could do what's shown below (We could make this easier to access down the line, if we feel the need)
print(f"Body marker names: {human.body.anatomical_structure.marker_names}")
print(f"Left hand marker names: {human.left_hand.anatomical_structure.marker_names}")


Body marker names: ['nose', 'left_eye_inner', 'left_eye', 'left_eye_outer', 'right_eye_inner', 'right_eye', 'right_eye_outer', 'left_ear', 'right_ear', 'mouth_left', 'mouth_right', 'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 'left_wrist', 'right_wrist', 'left_pinky', 'right_pinky', 'left_index', 'right_index', 'left_thumb', 'right_thumb', 'left_hip', 'right_hip', 'left_knee', 'right_knee', 'left_ankle', 'right_ankle', 'left_heel', 'right_heel', 'left_foot_index', 'right_foot_index', 'head_center', 'neck_center', 'trunk_center', 'hips_center']
Left hand marker names: ['wrist', 'thumb_cmc', 'thumb_mcp', 'thumb_ip', 'thumb_tip', 'index_finger_mcp', 'index_finger_pip', 'index_finger_dip', 'index_finger_tip', 'middle_finger_mcp', 'middle_finger_pip', 'middle_finger_dip', 'middle_finger_tip', 'ring_finger_mcp', 'ring_finger_pip', 'ring_finger_dip', 'ring_finger_tip', 'pinky_mcp', 'pinky_pip', 'pinky_dip', 'pinky_tip']


To add data from a numpy, use the `from_tracked_points_numpy` function, which splits up the data according to the information defined in the model info. 
Note that now in the printed aspect below, you can see a new trajectory `3d_xyz` added. The reason I called it `3d_xyz` is because that's the name that gets tagged to that Trajectory when saving out the file later.

In [38]:
human.from_tracked_points_numpy(tracked_points_numpy_array=data)
pprint([human.body])

Calculating virtual markers: ['head_center', 'neck_center', 'trunk_center', 'hips_center']
[Aspect: body
  Anatomical Structure:
  33 tracked points
  4 virtual markers
  22 segments
  14 center of mass definitions
  16 joint hierarchies
  Trajectories: 1 trajectories: ['3d_xyz']
  Metadata: : {'tracker_type': 'mediapipe'}

]


Here's how you can currently access the data. I think it's a little unwieldy and could be easier - but that's something to look at later. Below that you can also see how to get your data both as a numpy array and a pandas dataframe. 

In [39]:
print(human.body.trajectories['3d_xyz'].data)

{'nose': array([[ -584.25246118, -1188.90613348,  1866.26252851],
       [ -589.46152046, -1163.65964843,  1826.09886942],
       [ -589.1825987 , -1149.85056323,  1795.93403936],
       ...,
       [           nan,            nan,            nan],
       [           nan,            nan,            nan],
       [           nan,            nan,            nan]]), 'left_eye_inner': array([[ -575.12680531, -1202.53116076,  1805.62711625],
       [ -580.00204398, -1181.32111421,  1772.66176873],
       [ -577.22644514, -1165.22818768,  1745.08861151],
       ...,
       [           nan,            nan,            nan],
       [           nan,            nan,            nan],
       [           nan,            nan,            nan]]), 'left_eye': array([[ -561.68916378, -1186.02167732,  1790.86783741],
       [ -567.20045757, -1165.53239215,  1756.59792666],
       [ -562.46878092, -1151.84546508,  1730.89247207],
       ...,
       [           nan,            nan,            nan],
       [ 

In [40]:
print(f"Data as numpy: {human.body.trajectories['3d_xyz'].as_numpy}")

Data as numpy: [[[ -584.25246118 -1188.90613348  1866.26252851]
  [ -575.12680531 -1202.53116076  1805.62711625]
  [ -561.68916378 -1186.02167732  1790.86783741]
  ...
  [ -570.97071277  -847.32593025  1861.97617231]
  [ -561.5319131   -563.10438862  2020.40720672]
  [ -552.09311343  -278.88284699  2178.83824113]]

 [[ -589.46152046 -1163.65964843  1826.09886942]
  [ -580.00204398 -1181.32111421  1772.66176873]
  [ -567.20045757 -1165.53239215  1756.59792666]
  ...
  [ -559.60017119  -832.88736892  1830.73379816]
  [ -551.59994125  -541.01504886  2003.2166503 ]
  [ -543.59971131  -249.14272881  2175.69950244]]

 [[ -589.1825987  -1149.85056323  1795.93403936]
  [ -577.22644514 -1165.22818768  1745.08861151]
  [ -562.46878092 -1151.84546508  1730.89247207]
  ...
  [ -532.62145645  -814.70499902  1796.0181572 ]
  [ -533.009481    -524.6044414   1975.08750309]
  [ -533.39750556  -234.50388377  2154.15684897]]

 ...

 [[           nan            nan            nan]
  [           nan       

In [41]:
print(f"Data as dataframe: {human.body.trajectories['3d_xyz'].as_dataframe}")

Data as dataframe:        frame          keypoint           x            y            z
0          0              nose -584.252461 -1188.906133  1866.262529
1          0    left_eye_inner -575.126805 -1202.531161  1805.627116
2          0          left_eye -561.689164 -1186.021677  1790.867837
3          0    left_eye_outer -549.635370 -1168.348818  1774.846871
4          0   right_eye_inner -609.286370 -1224.678701  1824.191044
...      ...               ...         ...          ...          ...
40991   1107  right_foot_index         NaN          NaN          NaN
40992   1107       head_center         NaN          NaN          NaN
40993   1107       neck_center         NaN          NaN          NaN
40994   1107      trunk_center         NaN          NaN          NaN
40995   1107       hips_center         NaN          NaN          NaN

[40996 rows x 5 columns]


There's also some basic methods that I wrote into the Actor class that you can use to get marker or frame data. Those are demonstrated below.

While I think the way it's written is necessary for the Actor class, where we don't have a strong sense of what might be added to it, I don't think this method is ideal for the Human class. Too many strings involved. So it would probably be worth giving the human class additional methods to do the below functions.

In [42]:
print(human.get_marker_data(aspect_name= 'body', type = '3d_xyz', marker_name= 'left_heel'))
print(human.get_frame(aspect_name='body', type = '3d_xyz', frame_number=100))

[[-382.37437325  491.19936352 2670.4658261 ]
 [-447.57016682  463.4687682  2430.23763542]
 [-401.20027763  476.8665362  2290.70911599]
 ...
 [          nan           nan           nan]
 [          nan           nan           nan]
 [          nan           nan           nan]]
{'nose': array([-363.19744989, -904.32466624, 1412.39634963]), 'left_eye_inner': array([-360.7122946 , -920.12305488, 1369.49008208]), 'left_eye': array([-355.51463719, -908.59999488, 1355.03227349]), 'left_eye_outer': array([-349.70588642, -897.783565  , 1342.01736955]), 'right_eye_inner': array([-374.41393893, -953.07317705, 1409.82463033]), 'right_eye': array([-380.12619353, -961.87013385, 1420.35149925]), 'right_eye_outer': array([-387.37954099, -968.51605963, 1427.74761235]), 'left_ear': array([-340.73869748, -861.19796066, 1322.49458474]), 'right_ear': array([-398.04038958, -946.8901882 , 1426.98827259]), 'mouth_left': array([-351.66479001, -846.21260581, 1405.11441594]), 'mouth_right': array([-369.15238098, 

The following functions are what exist in our anatomical pipeline at the moment: calculating center of mass and enforcing rigid bones. For both, after the data is calculated, they get added as Trajectories to the relevant Aspect

In [43]:
for aspect in human.aspects.values():
    if aspect.anatomical_structure.center_of_mass_definitions:
        print('Calculating center of mass for aspect:', aspect.name)
        total_body_com, segment_com = calculate_center_of_mass_from_trajectory(aspect.trajectories['3d_xyz'], aspect.anatomical_structure.center_of_mass_definitions)

        aspect.add_total_body_center_of_mass(total_body_center_of_mass=total_body_com)
        aspect.add_segment_center_of_mass(segment_center_of_mass=segment_com)
    
    else:
        print('Skipping center of mass calculation for aspect:', aspect.name)

Calculating center of mass for aspect: body
Skipping center of mass calculation for aspect: face
Skipping center of mass calculation for aspect: left_hand
Skipping center of mass calculation for aspect: right_hand


In [44]:
for aspect in human.aspects.values():
    if aspect.anatomical_structure.joint_hierarchy:
        print('Enforcing rigid bones for aspect:', aspect.name)
        rigid_bones = enforce_rigid_bones_from_trajectory(aspect.trajectories['3d_xyz'], aspect.anatomical_structure.joint_hierarchy)

        aspect.add_trajectory(name = 'rigid_3d_xyz',
                                data = rigid_bones,
                                marker_names = aspect.anatomical_structure.marker_names)
    else:
        print('Skipping rigid bones enforcement for aspect:', aspect.name)

Enforcing rigid bones for aspect: body
Skipping rigid bones enforcement for aspect: face
Skipping rigid bones enforcement for aspect: left_hand
Skipping rigid bones enforcement for aspect: right_hand


Below you can see extra trajectories added to the body after running the above functions

In [45]:
for aspect in human.aspects.values():
    pprint({aspect.name: aspect.trajectories})

{'body': {'3d_xyz': <skellymodels.experimental.model_redo.models.trajectory.Trajectory object at 0x000001C5054FAB50>,
          'rigid_3d_xyz': <skellymodels.experimental.model_redo.models.trajectory.Trajectory object at 0x000001C506913A10>,
          'segment_com': <skellymodels.experimental.model_redo.models.trajectory.Trajectory object at 0x000001C506913F90>,
          'total_body_com': <skellymodels.experimental.model_redo.models.trajectory.Trajectory object at 0x000001C506913E50>}}
{'face': {'3d_xyz': <skellymodels.experimental.model_redo.models.trajectory.Trajectory object at 0x000001C50546D110>}}
{'left_hand': {'3d_xyz': <skellymodels.experimental.model_redo.models.trajectory.Trajectory object at 0x000001C50522BD50>}}
{'right_hand': {'3d_xyz': <skellymodels.experimental.model_redo.models.trajectory.Trajectory object at 0x000001C5069B5010>}}


These are our saving functions. They leverage the `as_numpy` and `as_dataframe` properties that each Trajectory has. We basically iterate over each Trajectory in an Aspect, and save it out, using the metadata stored within each aspect/trajectory to dynamically build the file names. The last function builds the 'big' CSV we have with all the data.

In [46]:
from skellymodels.experimental.model_redo.managers.actor import Actor
import pandas as pd

def save_out_numpy_data(actor:Actor):
    for aspect in actor.aspects.values():
        for trajectory in aspect.trajectories.values():
            print('Saving out numpy:', aspect.metadata['tracker_type'], aspect.name, trajectory.name)
            np.save(f"{aspect.metadata['tracker_type']}_{aspect.name}_{trajectory.name}.npy", trajectory.data)

def save_out_csv_data(actor:Actor):
    for aspect in actor.aspects.values():
        for trajectory in aspect.trajectories.values():
            print('Saving out CSV:', aspect.metadata['tracker_type'], aspect.name, trajectory.name)
            trajectory.as_dataframe.to_csv(f"{aspect.metadata['tracker_type']}_{aspect.name}_{trajectory.name}.csv")

def save_out_all_data_csv(actor: Actor):
    all_data = []

    # Loop through aspects and trajectories
    for aspect_name, aspect in actor.aspects.items():
        for trajectory_name, trajectory in aspect.trajectories.items():
            if trajectory_name == '3d_xyz':
                # Get tidy DataFrame for the trajectory
                trajectory_df = trajectory.as_dataframe
                
                # Add metadata column for model
                trajectory_df['model'] = f"{aspect.metadata['tracker_type']}_{aspect_name}"
                
                # Append DataFrame to the list
                all_data.append(trajectory_df)

    # Combine all DataFrames into one
    big_df = pd.concat(all_data, ignore_index=True)

    # Sort by frame and then by model
    big_df = big_df.sort_values(by=['frame', 'model']).reset_index(drop=True)

    # Save the result to CSV
    big_df.to_csv('freemocap_data_by_frame.csv', index=False)
    print("Data successfully saved to 'freemocap_data_by_frame.csv'")


And these three functions are all we need to save out similar/almost equivalent data to what we currently save out for freemocap in the output data folder

In [47]:
save_out_numpy_data(actor = human)
save_out_csv_data(actor = human)
save_out_all_data_csv(actor = human)


Saving out numpy: mediapipe body 3d_xyz
Saving out numpy: mediapipe body total_body_com
Saving out numpy: mediapipe body segment_com
Saving out numpy: mediapipe body rigid_3d_xyz
Saving out numpy: mediapipe face 3d_xyz
Saving out numpy: mediapipe left_hand 3d_xyz
Saving out numpy: mediapipe right_hand 3d_xyz
Saving out CSV: mediapipe body 3d_xyz
Saving out CSV: mediapipe body total_body_com
Saving out CSV: mediapipe body segment_com
Saving out CSV: mediapipe body rigid_3d_xyz
Saving out CSV: mediapipe face 3d_xyz
Saving out CSV: mediapipe left_hand 3d_xyz
Saving out CSV: mediapipe right_hand 3d_xyz
Data successfully saved to 'freemocap_data_by_frame.csv'
